이미지를 4분할해서 각각을 640*360

In [16]:
import os
import natsort
import cv2

# 원본 이미지 폴더와 출력 폴더 설정
image_root = "/mnt/hdd_6tb/seungeun/HuNature/data/image/plate"
sorted_image_folder_list = natsort.natsorted(os.listdir(image_root))
output_folder = "/mnt/hdd_6tb/bill0914/processed_quarter/image"
os.makedirs(output_folder, exist_ok=True)  # 출력 폴더 생성

idx = 1  # 이미지 고유 번호

for subfolder in sorted_image_folder_list:
    subfolder_path = os.path.join(image_root, subfolder)
    image_file_list = natsort.natsorted(os.listdir(subfolder_path))
    
    for image in image_file_list:
        image_path = os.path.join(subfolder_path, image)
        
        # 이미지 읽기
        img = cv2.imread(image_path)
        if img is None:
            print(f"이미지 {image_path}를 열 수 없습니다.")
            continue
        
        h, w = img.shape[:2]  # 이미지 높이와 너비 가져오기
        
        # 이미지 분할
        img2 = img[h//2:h, 0:w//2]  # 좌하단
        img3 = img[h//2:h, w//2:w]  # 우하단
        img0 = img[0:h//2, 0:w//2]  # 좌상단
        img1 = img[0:h//2, w//2:w]  # 우상단
        
        # 분할된 이미지를 저장 (640x640으로 리사이즈)
        for j, split_img in enumerate([img0, img1, img2, img3]):
            resized_img = cv2.resize(split_img, (640,360))  # 640x640으로 리사이즈
            output_path = os.path.join(output_folder, f"img_{idx}_{j}.png")
            cv2.imwrite(output_path, resized_img)
        
        idx += 1

print("4분할된 이미지 저장이 완료되었습니다.")


4분할된 이미지 저장이 완료되었습니다.


라벨 데이터를 4분할로 저장하자

In [17]:
import natsort
import os
import json

input_folder = "/mnt/hdd_6tb/seungeun/HuNature/data/label/plate"
output_folder = "/mnt/hdd_6tb/bill0914/processed_quarter/label"
input_folder_list = natsort.natsorted(os.listdir(input_folder))  # [cr06, cr11, cr12, cr13, cr14]

def bbox_division(liss, W, H):
    x, y, w, h = liss
    x_new = x + w / 2
    y_new = y + h / 2
    if x_new <= W / 2 and y_new <= H / 2:
        return x_new, y_new, w, h, 0
    if x_new <= W / 2 and y_new >= H / 2:
        y_new = y_new - H/2
        return x_new, y_new, w, h, 2
    if x_new >= W / 2 and y_new <= H / 2:
        x_new = x_new - W/2
        return x_new, y_new, w, h, 1
    if x_new >=W/2 and y_new >=H/2:
        x_new = x_new - W/2
        y_new = y_new - H/2
        return x_new, y_new, w, h, 3
    else:
        print('이상하다')
k = 1
for subfolder in input_folder_list:
    subfolder_path = os.path.join(input_folder, subfolder)
    for jsons in natsort.natsorted(os.listdir(subfolder_path)):
        json_path = os.path.join(subfolder_path, jsons)
        with open(json_path, 'r') as file:
            data = json.load(file)
            width, height = map(int, data['Raw_Data_Info']['resolution'].split(','))

        # 데이터 중복 제거를 위한 저장소    
        file_data = {}

        for annotation in data['Learning_Data_Info']['annotations']:
            for plate in annotation.get('license_plate', []):  # 각 bbox 처리
                bbox = plate.get('bbox')
                x_new, y_new, w, h, idx = bbox_division(bbox, width, height)
                file_name = f"img_{k}_{idx}.txt"
                
                # 중복 방지: 파일 데이터 저장소에 추가
                if file_name not in file_data:
                    file_data[file_name] = set()
                file_data[file_name].add(f"{0} {x_new*2 / width} {y_new*2 / height} {w *2/ width} {h *2/ height}")

        # 파일 기록 (중복 제거된 데이터만 기록)
        for file_name, content in file_data.items():
            with open(os.path.join(output_folder, file_name), 'w') as f:  # 'w'로 파일 덮어쓰기
                f.write("\n".join(content) + "\n")

        k += 1


빈줄 다 없애자


In [18]:
import os

def remove_trailing_newline(input_file, output_file):
    with open(input_file, 'r') as infile:
        lines = infile.readlines()

    # 각 줄에서 개행 문자를 제거하고 마지막 줄에만 개행 추가
    cleaned_lines = [line.rstrip("\n") for line in lines]

    # 파일에 저장 (마지막 줄 뒤에 개행 없이 저장)
    with open(output_file, 'w') as outfile:
        outfile.write("\n".join(cleaned_lines))

# 예시 사용법
input_folder = "/mnt/hdd_6tb/bill0914/processed_quarter/label"
output_folder = "/mnt/hdd_6tb/bill0914/processed_quarter/label_no"

os.makedirs(output_folder, exist_ok=True)

# 폴더 내 모든 파일에 대해 마지막 줄의 불필요한 개행 제거
for file_name in natsort.natsorted(os.listdir(input_folder)):
    input_file_path = os.path.join(input_folder, file_name)
    output_file_path = os.path.join(output_folder, file_name)

    remove_trailing_newline(input_file_path, output_file_path)
    print(f"Processed {file_name}")


Processed img_1_2.txt
Processed img_2_2.txt
Processed img_2_3.txt
Processed img_3_3.txt
Processed img_4_3.txt
Processed img_5_0.txt
Processed img_5_2.txt
Processed img_5_3.txt
Processed img_6_0.txt
Processed img_6_1.txt
Processed img_6_2.txt
Processed img_6_3.txt
Processed img_7_1.txt
Processed img_7_2.txt
Processed img_7_3.txt
Processed img_8_3.txt
Processed img_9_3.txt
Processed img_10_2.txt
Processed img_10_3.txt
Processed img_11_2.txt
Processed img_11_3.txt
Processed img_12_2.txt
Processed img_13_3.txt
Processed img_14_1.txt
Processed img_14_2.txt
Processed img_14_3.txt
Processed img_15_0.txt
Processed img_15_1.txt
Processed img_15_2.txt
Processed img_15_3.txt
Processed img_16_2.txt
Processed img_17_2.txt
Processed img_18_1.txt
Processed img_18_2.txt
Processed img_18_3.txt
Processed img_19_2.txt
Processed img_19_3.txt
Processed img_20_0.txt
Processed img_20_2.txt
Processed img_20_3.txt
Processed img_21_0.txt
Processed img_21_2.txt
Processed img_21_3.txt
Processed img_22_3.txt
Proce

이제 이미지와 라벨을 일치시켜보자

In [1]:
import os
import natsort

# 라벨 파일 폴더와 이미지 파일 폴더 경로
label_folder = "/mnt/hdd_6tb/bill0914/processed_quarter/label_no"
image_folder = "/mnt/hdd_6tb/bill0914/processed_quarter/image"

# 라벨 이름 추출
label_list = []
for file in natsort.natsorted(os.listdir(label_folder)):
    name, _ = os.path.splitext(file)
    label_list.append(name.split("img_")[1])  # "img_" 이후의 이름 저장

# 이미지 이름 추출
image_list = []
for file in natsort.natsorted(os.listdir(image_folder)):
    name, ext = os.path.splitext(file)
    image_list.append(name.split("img_")[1])  # "img_" 이후의 이름 저장

print(f"총 이미지 파일 개수: {len(image_list)}")
print(f"총 라벨 파일 개수: {len(label_list)}")

# 라벨에 없는 이미지를 삭제
deleted_count = 0
for file in natsort.natsorted(os.listdir(image_folder)):
    name, ext = os.path.splitext(file)
    image_id = name.split("img_")[1]
    
    if image_id not in label_list:
        file_path = os.path.join(image_folder, file)
        os.remove(file_path)  # 파일 삭제
        deleted_count += 1
        print(f"삭제: {file_path}")

print(f"삭제된 이미지 파일 개수: {deleted_count}")
print(f"남은 이미지 파일 개수: {len(os.listdir(image_folder))}")



총 이미지 파일 개수: 47548
총 라벨 파일 개수: 25090
삭제: /mnt/hdd_6tb/bill0914/processed_quarter/image/img_1_0.png
삭제: /mnt/hdd_6tb/bill0914/processed_quarter/image/img_1_1.png
삭제: /mnt/hdd_6tb/bill0914/processed_quarter/image/img_1_3.png
삭제: /mnt/hdd_6tb/bill0914/processed_quarter/image/img_2_0.png
삭제: /mnt/hdd_6tb/bill0914/processed_quarter/image/img_2_1.png
삭제: /mnt/hdd_6tb/bill0914/processed_quarter/image/img_3_0.png
삭제: /mnt/hdd_6tb/bill0914/processed_quarter/image/img_3_1.png
삭제: /mnt/hdd_6tb/bill0914/processed_quarter/image/img_3_2.png
삭제: /mnt/hdd_6tb/bill0914/processed_quarter/image/img_4_0.png
삭제: /mnt/hdd_6tb/bill0914/processed_quarter/image/img_4_1.png
삭제: /mnt/hdd_6tb/bill0914/processed_quarter/image/img_4_2.png
삭제: /mnt/hdd_6tb/bill0914/processed_quarter/image/img_5_1.png
삭제: /mnt/hdd_6tb/bill0914/processed_quarter/image/img_7_0.png
삭제: /mnt/hdd_6tb/bill0914/processed_quarter/image/img_8_0.png
삭제: /mnt/hdd_6tb/bill0914/processed_quarter/image/img_8_1.png
삭제: /mnt/hdd_6tb/bill0914/process

data폴더로 split하기 75:25

In [2]:
import os
import shutil
import random

# 원본 디렉토리
label_dir = "/mnt/hdd_6tb/bill0914/processed_quarter/label_no"
image_dir = "/mnt/hdd_6tb/bill0914/processed_quarter/image"

# 복사할 목표 디렉토리
train_label_dir = "/mnt/hdd_6tb/bill0914/processed_quarter/data/train/labels"
val_label_dir = "/mnt/hdd_6tb/bill0914/processed_quarter/data/val/labels"
train_image_dir = "/mnt/hdd_6tb/bill0914/processed_quarter/data/train/images"
val_image_dir = "/mnt/hdd_6tb/bill0914/processed_quarter/data/val/images"

# 디렉토리 생성 (없으면 생성)
os.makedirs(train_label_dir, exist_ok=True)
os.makedirs(val_label_dir, exist_ok=True)
os.makedirs(train_image_dir, exist_ok=True)
os.makedirs(val_image_dir, exist_ok=True)

# 라벨 파일 목록 가져오기
label_files = [f for f in os.listdir(label_dir) if f.endswith('.txt')]

# 라벨 파일 섞기 (랜덤 셔플)
random.shuffle(label_files)

# 75%는 train, 25%는 val로 분할
split_index = int(len(label_files) * 0.75)
train_files = label_files[:split_index]
val_files = label_files[split_index:]

# 복사 함수
def copy_files(file_list, src_label_dir, src_image_dir, dest_label_dir, dest_image_dir):
    for label_file in file_list:
        # 라벨 파일 경로
        src_label_path = os.path.join(src_label_dir, label_file)
        dest_label_path = os.path.join(dest_label_dir, label_file)
        
        # 이미지 파일 경로
        image_file = label_file.replace('.txt', '.png')  # 라벨 파일명을 이미지 파일명으로 변환
        src_image_path = os.path.join(src_image_dir, image_file)
        dest_image_path = os.path.join(dest_image_dir, image_file)
        
        # 라벨 복사
        shutil.copy(src_label_path, dest_label_path)
        
        # 이미지가 존재하면 복사
        if os.path.exists(src_image_path):
            shutil.copy(src_image_path, dest_image_path)
        else:
            print(f"이미지 파일 {image_file}가 존재하지 않습니다.")

# Train 데이터 복사
copy_files(train_files, label_dir, image_dir, train_label_dir, train_image_dir)

# Val 데이터 복사
copy_files(val_files, label_dir, image_dir, val_label_dir, val_image_dir)

print(f"Train 라벨 개수: {len(train_files)}")
print(f"Val 라벨 개수: {len(val_files)}")


Train 라벨 개수: 18817
Val 라벨 개수: 6273


In [2]:
dir="/mnt/hdd_6tb/bill0914/processed_quarter/test/bbox_counts"
import os
from natsort import natsorted
num=0
for i in natsorted(os.listdir(dir)):
    with open(os.path.join(dir,i),'r') as f:
        c = f.read()
        c = int(c)
        num+=c
print(num)        


1453
